# Read html to get TABLE 1

In [1]:
import os
#import requests 
import pandas as pd
import bs4
from bs4 import BeautifulSoup
#from lxml import html
from html_table_extractor.extractor import Extractor
import numpy as np

In [2]:
# setup working directory
os.chdir("/temp_project/ukbb/data")
path="/temp_project/ukbb/ukb22598.html"

f = open(path)      # open html file
#soup = BeautifulSoup(f,"lxml")
soup = bs4.BeautifulSoup(f, 'html.parser')
f.close()

extractor = Extractor(soup)
extractor.parse()
#extractor.return_list()[2:5]   #table 1 starts at row 2
#extractor.return_list()[8777:8780]   #table 1 ends at row 8780
col_labels=extractor.return_list()[2]   #colnames for table 1 

tab1=extractor.return_list()[3:8780] 
df_tab1=pd.DataFrame(tab1[0:],columns=col_labels)
df_tab1.to_csv('ukb22598_html_tab1.csv',index=False)

## Add TIER info
tier_path='/temp_project/ukbb/load_Variable_Triage.csv'
tier_table=pd.read_csv(tier_path)
tier_table.head()
tier_table_for_merge=tier_table.drop(columns='Column')
tier_table_for_merge.head()


df_tab1_tier=pd.merge(df_tab1,tier_table_for_merge, on='UDI', how='left')


df_tab1ex=df_tab1_tier
df_des=df_tab1ex[['Description']]


## Extract data-coding & levels info
pdf=df_des.Description.str.partition('Uses data-coding ')

pdf.head()
ppdf=pdf[[2]]
ppdf.columns=['dc']
pppdf=ppdf.dc.str.partition(' ')
dccol=pppdf[[0]]
dccol.columns=['data_coding']


catcol=df_des.Description.str.partition('comprises ')

catcol.head()
catcol2=catcol[[2]]
catcol2.columns=['cat']
catcol3=catcol2.cat.str.partition(' ')
catcol3
catcol4=catcol3[[0]]
catcol4.columns=['CatLv']


df_tab1ex['DC']=dccol
df_tab1ex['CatLv']=catcol4
df_tab1ex.CatLv.describe()

dfchk=df_tab1ex.copy()


## break UDI into two parts: filed id as udip1 and x.x as udip2
df_udi_List = df_tab1['UDI'].tolist()
udi_long=[i.split('-', 1)[0] for i in df_udi_List]
udi_long2=[i.split('-', 1)[-1] for i in df_udi_List]
print(len(udi_long),len(udi_long2),len(df_udi_List),len(dfchk))
#udi_long2
#udi_long[0]=''
#udi_long2[0]=''

dfchk['udip1']=udi_long
dfchk['udip2']=udi_long2
#dfchk0=dfchk

## break udip2 into two parts: m.n--> udip2m udip2n
df_udip2_List = dfchk['udip2'].tolist()
udip2_m=[i.split('.', 1)[0] for i in df_udip2_List]
udip2_n=[i.split('.', 1)[-1] for i in df_udip2_List]
udip2_m[0]=-999
udip2_n[0]=-999

udip2_m = [int(i) for i in udip2_m]
udip2_n = [int(i) for i in udip2_n]

dfchk['udip2m']=udip2_m
dfchk['udip2n']=udip2_n


# add missing count / pct
dfred_mischk=dfchk['Count']
len(dfred_mischk)
type(dfred_mischk)
#dfred_mischk
#dfred_mischk.loc.Column=dfred_t0r0.Column.astype(int)
dfred_mischk_col=dfred_mischk.astype(int)
type(dfred_mischk_col)
dfchk['miss_ct']=502602-dfred_mischk_col
dfchk['miss_pt']=(502602-dfred_mischk_col)/502602
dfchk['obs_ct']=dfred_mischk_col
dfchk['obs_pt']=dfred_mischk_col/502602
2/502602*100

os.chdir("/temp_project/ukbb/data")
dfchk.to_csv('ukb22598_html_tab1_adv.csv',index=False)

8777 8777 8777 8777


# Based on TABLE 1: extract i=0 variables by variable type

if the folder 

/temp_project/ukbb/data/i0/var_xxx 

(xxx is a variable abbv.) is not empty, no need to run the data generating code at folder 

/temp_project/ukbb/data/i0/code

Otherwise, run those codes first

In [3]:
# /

# Generate i=0 comphensive all variable description table 

In [4]:
import os
import pandas as pd


In [5]:
os.chdir("/temp_project/ukbb/data")
df_tab1_adv=pd.read_csv('ukb22598_html_tab1_adv.csv')
df_tab1_adv['DC'] = df_tab1_adv['DC'].fillna('NaN')
df_tab1_adv['CatLv'] = df_tab1_adv['CatLv'].fillna('NaN')

In [6]:

os.chdir('/temp_project/ukbb/data/i0/varInstanceInfo')
## List of unique variables for all types
uniqchk=df_tab1_adv.groupby('udip1').head(1).reset_index(drop=True)

#uniqchk[uniqchk['udip2m']==2]  ## start from -2.x has 11 unique counts, in total 26 variables
uniqchk_sta_not0x=uniqchk[uniqchk['udip2m']!=0] 
list_uniqchk_sta_not0x=uniqchk_sta_not0x['udip1'].tolist()
list_uniqchk_sta_not0x       ## eid + 11 unique var starts -2.x, that is unique 12 vars, in total 27 variable

uniqchk[uniqchk['udip2m']==0].udip2n.value_counts()   
## start from -0.x has 1021 unique counts (in total 8750 variables), where start as 
## -0.0 has 1001
## -0.1 has 20

uniqchk_sta0x=uniqchk[uniqchk['udip2m']==0]   ## 1021 unique var starts like '-0.x'
uniqchk_sta01=uniqchk_sta0x[uniqchk_sta0x['udip2n']==1] ## 20 unique var starts like '-0.1'
uniqchk_sta00=uniqchk_sta0x[uniqchk_sta0x['udip2n']==0] ## 1001 unique var starts like '-0.0'

list_uniqchk_sta01=uniqchk_sta01['udip1'].tolist()
list_uniqchk_sta00=uniqchk_sta00['udip1'].tolist()

df_stall=df_tab1_adv[~df_tab1_adv.udip1.isin(list_uniqchk_sta_not0x)]    # len=8570 = 8777-27


df_sta01=df_stall[df_stall.udip1.isin(list_uniqchk_sta01)]
df_sta00=df_stall[df_stall.udip1.isin(list_uniqchk_sta00)]
df_sta2x=df_tab1_adv[df_tab1_adv.udip1.isin(list_uniqchk_sta_not0x)]

#len(df_sta01)   ## 232 vars
#len(df_sta00)   ## 8518 vars
#len(df_sta2x)   ## 27 vars

df_sta01_for_icount=df_sta01[df_sta01['udip2n']==1]
df_sta00_for_icount=df_sta00[df_sta00['udip2n']==0]

#len(df_sta01_for_icount)   ## 30
#len(df_sta00_for_icount)   ## 2529

field_i3or5_sta01=df_sta01_for_icount.udip1.value_counts()
field_i3or5_sta01.to_csv('afield_i3or5_sta01.csv')

field_i3or5_sta00=df_sta00_for_icount.udip1.value_counts()
field_i3or5_sta00.to_csv('afield_i3or5_sta00.csv')

field_i3or5_sta01=pd.read_csv('afield_i3or5_sta01.csv',header=None)
field_i3or5_sta01.columns=['field','instance_type']

field_i3or5_sta00=pd.read_csv('afield_i3or5_sta00.csv',header=None)
field_i3or5_sta00.columns=['field','instance_type']

field_i3or5_sta01
field_i3or5_to_create_ind=pd.merge(field_i3or5_sta00, field_i3or5_sta01, on="field", how='outer')
field_i3or5_to_create_ind['instance_type']=field_i3or5_to_create_ind.drop('field',axis=1).max(axis=1)
#len(field_i3or5_sta01)   ## 20
#len(field_i3or5_sta00)   ## 1001
#len(field_i3or5_to_create_ind)   ## 1021
field_i3or5_to_ind_pre=pd.get_dummies(field_i3or5_to_create_ind, prefix='instance_type_ind', columns=['instance_type'])
field_i3or5_to_ind=field_i3or5_to_ind_pre[['field','instance_type_ind_3.0','instance_type_ind_5.0','instance_type_ind_30.0','instance_type_ind_32.0']]
field_i3or5_to_ind.columns=['udip1','instance_type_ind3','instance_type_ind5','instance_type_ind30','instance_type_ind32']
field_i3or5_to_ind.head()

df1=df_tab1_adv.copy()
df2=field_i3or5_to_ind

dtype = dict(udip1=str)
#df_tab1_adv2=df1.astype(dtype).merge(df2.astype(dtype), 'left')
df_tab1_adv2=pd.merge(df1.astype(dtype),df2.astype(dtype), on='udip1', how='left')
df_tab1_adv2.to_csv('/temp_project/ukbb/data/ukb22598_html_tab1_comp.csv',index=False)

In [22]:
df_tab1_adv2.head(2)
df_tab1_adv2[df_tab1_adv2.udip1=='6142'][['Column','UDI','Count','Type','Description','DC']]


,Column,UDI,Count,Type,Description,DC
2609,2609,6142-0.0,501731,Categorical (multiple),Current employment statusUses data-coding 1002...,100295
2610,2610,6142-0.1,39551,Categorical (multiple),Current employment statusUses data-coding 1002...,100295
2611,2611,6142-0.2,6159,Categorical (multiple),Current employment statusUses data-coding 1002...,100295
2612,2612,6142-0.3,511,Categorical (multiple),Current employment statusUses data-coding 1002...,100295
2613,2613,6142-0.4,35,Categorical (multiple),Current employment statusUses data-coding 1002...,100295
2614,2614,6142-0.5,3,Categorical (multiple),Current employment statusUses data-coding 1002...,100295
2615,2615,6142-0.6,1,Categorical (multiple),Current employment statusUses data-coding 1002...,100295
2616,2616,6142-1.0,20338,Categorical (multiple),Current employment statusUses data-coding 1002...,100295
2617,2617,6142-1.1,1774,Categorical (multiple),Current employment statusUses data-coding 1002...,100295
2618,2618,6142-1.2,332,Categorical (multiple),Current employment statusUses data-coding 1002...,100295


# Generate i=0 comphensive unique var table

In [8]:
import os
import pandas as pd
import numpy as np

## check categorical variable frequence for each level 
def freq_chk(dataframe,var='',output=True):
    ctab=pd.crosstab(dataframe[var], columns='count')
    if output==True:
        print(ctab)
    else:
        pass
    return;

# .value_counts()

In [9]:
import os
import pandas as pd

### The following code need to run 5 min-ish
### caluculat missing percentage for certain variable type
def missing_cal(var_tp):
    
    var_tp_path='/temp_project/ukbb/data/i0/var_'+str(var_tp)
    os.chdir(str(var_tp_path))
    listvar_i0=os.listdir() 
    #chkNfile='/temp_project/ukbb/data/i0/var_con/'+str(listvar_i0[0])
    #chkN=pd.read_csv(chkNfile)
    #N=len(chkN)
    N=502602
    
    mis_ct=[]
    
    for i in listvar_i0:
        pre,typ,fidx=i.partition(str(var_tp))
        fid,part,inst=fidx.partition('i')
        if 'vec' not in i:
            chk=pd.read_csv(str(i))
            df_pre=chk.drop(['eid'],axis=1).describe()
            df_pre.columns=[str(fid)]
            df_t=df_pre.T
            df_t['fid']=df_t.index
            df_tc=df_t[['fid','count']]
        elif 'vec' in i:
            chk=pd.read_csv(str(i),sep='\t')
            df_pre=chk.drop(['eid'],axis=1).describe()
            df_pre.columns=[str(fid)]
            df_t=df_pre.T
            df_t['fid']=df_t.index
            df_tc=df_t[['fid','count']]
        mis_ct.append(df_tc)
        
    mis_tab=pd.concat(mis_ct)
    mis_tab['obs_ct']=mis_tab['count'].astype(int)
    mis_tab['obs_pct']=mis_tab['count'].astype(int)/N
    mis_tab['miss_ct']=N-mis_tab['count'].astype(int)
    mis_tab['miss_pct']=(N-mis_tab['count'].astype(int))/N
    
    return mis_tab


var_type_list=['con','cur','dat','int','tex','tim','cas','cam']
#var_type_list=['cur','dat']
mistab={}
for i in var_type_list:
    out_file='mis_tab_'+str(i)
    mistab[out_file]=missing_cal(str(i))



miss_tab_i0=pd.concat(mistab)

os.chdir('/temp_project/ukbb/data/i0/varDesp')
miss_tab_i0.to_csv('miss_tab_i0.csv',index=None)


In [10]:
### unique fid table with missing information


os.chdir("/temp_project/ukbb/data")
df_tab1_adv=pd.read_csv('ukb22598_html_tab1_comp.csv')
df_tab1_adv['DC'] = df_tab1_adv['DC'].fillna('NaN')
df_tab1_adv['CatLv'] = df_tab1_adv['CatLv'].fillna('NaN')
## List of unique variables for all types
uniqchk=df_tab1_adv.groupby('udip1').head(1).reset_index(drop=True)

fid_i0_unique=uniqchk[uniqchk['udip2m']==0] 


d1=fid_i0_unique[['udip1','Type','Description','Tier','DC','CatLv','instance_type_ind3','instance_type_ind5','instance_type_ind30','instance_type_ind32']]

os.chdir('/temp_project/ukbb/data/i0/varDesp')
d2=pd.read_csv('miss_tab_i0.csv')

d1.columns=['fid','Type','Description','Tier','DC','CatLv','Ind_3instance','Ind_5instance','Ind_30instance','Ind_32instance']
d3=d2[['fid','obs_ct','obs_pct','miss_ct','miss_pct']]



dtype = dict(fid=str)
#df_tab1_adv2=df1.astype(dtype).merge(df2.astype(dtype), 'left')
df_tab1_i0_comp=pd.merge(d1.astype(dtype),d3.astype(dtype), on='fid', how='left')
df_tab1_i0_comp.to_csv('/temp_project/ukbb/data/i0/ukb22598_i0_comp.csv',index=False)



In [11]:
#df_tab1_i0_comp_slides_use=df_tab1_i0_comp.drop(['Ind_3instance','Ind_5instance','Ind_30instance','Ind_32instance'], axis=1).copy()
#df_tab1_i0_comp_slides_use.head(8)

In [12]:
#df_tab1_i0_comp_slides_use[df_tab1_i0_comp_slides_use.miss_pct==0]
#df_tab1_i0_comp_slides_use[df_tab1_i0_comp_slides_use.miss_pct==1]


In [13]:
df_tab1_i0_comp=pd.read_csv('/temp_project/ukbb/data/i0/ukb22598_i0_comp.csv')
df_tab1_i0_comp[df_tab1_i0_comp.fid==int(21)].Description.values[0]

'Weight methodUses data-coding 100261 comprises 5 Integer-valued members in a simple list.'

# load data by fid source code

In [43]:
import os
import pandas as pd
import numpy as np

def load_data_by_fid(fid):
    '''
    return a dataframe that has the eid and the 'fid' variable  
    '''
    df_tab1_i0_comp=pd.read_csv('/temp_project/ukbb/data/i0/ukb22598_i0_comp.csv')

    if int(fid) in df_tab1_i0_comp.fid.values.tolist():
        fid_num=fid
        
        var_description = df_tab1_i0_comp[df_tab1_i0_comp['fid']==int(fid_num)].Description.values[0]
        var_type=df_tab1_i0_comp[df_tab1_i0_comp['fid']==int(fid_num)].Type.values[0]

        var_type_list=['con','cur','dat','int','tex','tim','cas','cam']
        var_type_list_full=['Continuous','Curve','Date','Integer','Text','Time','Categorical (single)', 'Categorical (multiple)']

        path_p1='/temp_project/ukbb/data/i0/var_'

        if var_type in var_type_list_full:
            vtyp=var_type_list[var_type_list_full.index(var_type)]

        loadpath=path_p1+str(vtyp)+'/'
        os.chdir(path_p1+str(vtyp))
        list_folder=os.listdir() 

        pname1=str(vtyp)+str(fid_num)+'i0.csv'
        pname2='vec_'+str(vtyp)+str(fid_num)+'i0.csv'

        if pname1 in list_folder:
#            print('fid ' + str(fid_num) + ' is a single-measure '+str(var_type).lower()+' variable')
            print('fid ' + str(fid_num) + ' is a single-measure '+str(var_type).lower()+' variable, which is \n'+str(var_description))
            fpname=list_folder[list_folder.index(pname1)]
            df_load=pd.read_csv(loadpath+fpname)

        elif pname2 in list_folder:
#            print('fid ' + str(fid_num) + ' is a multiple-measure '+str(var_type).lower()+' variable')
            print('fid ' + str(fid_num) + ' is a single-measure '+str(var_type).lower()+' variable, which is \n'+str(var_description))

            fpname=list_folder[list_folder.index(pname2)]
            df_load=pd.read_csv(loadpath+fpname, sep='\t')
        return df_load
    
    else:
        print('fid not found, please try again')


In [ ]:
%%time 
dfc=load_data_by_fid(input('enter fid: '))
if dfc is not None:
    print(dfc.head())